In [1]:
import numpy as np
import time
from gridfs import GridFS
from pymongo import MongoClient
from PIL import Image

from padma.models.encoder.encoder import EncoderModel, SIZE

from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.utils.images import generate_batch, get_imagens_recortadas

db = MongoClient(host=MONGODB_URI)[DATABASE]
fs = GridFS(db)

encoder = EncoderModel()

Using TensorFlow backend.
2019-03-24 08:23:45,453 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/error.log
Fazendo log de atividade no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/access.log


In [6]:
batch_size = 10
limit = 100
TEMBBOX = {'metadata.contentType': 'image/jpeg',
          'metadata.predictions.bbox': {'$exists': True}}
dict_filtro = TEMBBOX
projection = ['metadata.predictions']
# dict_filtro['metadata.predictions.index'] = False

In [7]:
db['fs.files'].count_documents(dict_filtro)

33879

In [8]:
batch_gen = generate_batch(db, filtro=dict_filtro, projection=projection,
                           batch_size=batch_size, limit=limit)

In [9]:
X = np.zeros((batch_size, *SIZE, 1), dtype=np.float32)
_ids = []
y = []
s = time.time()
for batch, rows in batch_gen:
    if len(batch) == 0:
        break
    for i, (images, row) in enumerate(zip(batch, rows)):
        image_array = encoder.image_prepare(images[0])
        s0 = time.time()
        X[i, :, :, :] = image_array
        _ids.append(row['_id'])
    s1 = time.time()
    print('Montou X em %0.2f ' % (s1 - s0))
    indexes = encoder.model.predict(X)
    indexes = indexes.reshape(-1, 128).astype(np.float32)
    y.append(indexes)
    s2 = time.time()
    print('Fez predição em %s' % (s2 - s1))
    # print(indexes)
s3 = time.time()
elapsed = s3 - s
print('Tempo total: %s. Por imagem: %s' % (elapsed, elapsed / limit))
    

Montou X em 0.00 
Fez predição em 3.137857675552368
Montou X em 0.00 
Fez predição em 0.003420114517211914
Montou X em 0.00 
Fez predição em 0.0030679702758789062
Montou X em 0.00 
Fez predição em 0.003386974334716797
Montou X em 0.00 
Fez predição em 0.0033788681030273438
Montou X em 0.00 
Fez predição em 0.0033111572265625
Montou X em 0.00 
Fez predição em 0.0032911300659179688
Montou X em 0.00 
Fez predição em 0.003197908401489258
Montou X em 0.00 
Fez predição em 0.0033867359161376953
Montou X em 0.00 
Fez predição em 0.0035314559936523438
Tempo total: 4.903109788894653. Por imagem: 0.04903109788894653


In [10]:
indexes.shape

(10, 128)

In [11]:
ystack = np.vstack(y).astype(np.float32)
ystack.shape

(100, 128)

In [12]:
len(_ids)

100

In [13]:
# TODO: Fazer Update no MongoDB

In [23]:
for i, _id in enumerate(_ids):
    index_row = db.fs.files.find_one({'_id': _id})
    new_array = np.array(index_row['metadata']['predictions'][0]['index'])
    dif = sum(np.abs(new_array - ystack[i, :])) / sum(ystack[i, :])
    if dif > 0.003:
        print(dif)

0.0032280869425335215
0.003298138652494815
0.0059799631818744925
0.0032428158994950566
0.0042197219905472155
0.004423547686935164
0.005262609044349932
0.0032404724398046245
0.0031926974354299963
0.003296075120381471
0.0033044152694217278
0.003136928393766496
0.003494608393556311


In [17]:
ystack[1, :] 

array([0.        , 1.3295321 , 0.        , 0.62208045, 0.23506866,
       0.        , 0.        , 0.60089624, 0.        , 0.8367415 ,
       0.        , 0.14311688, 0.0850147 , 0.        , 0.        ,
       0.20331147, 0.        , 0.8332069 , 0.        , 0.15773489,
       0.09196942, 0.        , 0.        , 0.20543829, 0.        ,
       1.3008422 , 0.0869644 , 0.23300102, 0.6513468 , 0.        ,
       0.10353519, 0.18565348, 0.        , 1.5958971 , 0.        ,
       0.7624705 , 0.17550229, 0.        , 0.        , 0.51158774,
       0.        , 1.0940543 , 0.        , 0.23417398, 0.07671893,
       0.        , 0.        , 0.11120109, 0.        , 1.0807372 ,
       0.        , 0.23848516, 0.08987951, 0.        , 0.        ,
       0.10644001, 0.        , 1.3831366 , 0.1780239 , 0.2924728 ,
       0.60475403, 0.        , 0.06225286, 0.08638769, 0.        ,
       1.9126328 , 0.        , 0.8066144 , 0.15624855, 0.        ,
       0.        , 0.58040476, 0.        , 1.3734322 , 0.     

In [18]:
new_list = ystack[0, :].tolist()
index_row = db.fs.files.find_one({'_id': _ids[0]})
old_list = index_row['metadata']['predictions'][0]['index']
predictions = index_row['metadata']['predictions']

In [19]:
_ids[0]

ObjectId('5c5309281004b3779c37d3a2')

In [20]:
new_list

[0.0,
 1.2453491687774658,
 0.0,
 0.8244212865829468,
 0.0,
 0.0,
 0.0,
 0.38313350081443787,
 0.0,
 1.0050201416015625,
 0.0,
 0.10921116173267365,
 0.0787082314491272,
 0.0,
 0.0,
 0.2451401650905609,
 0.0,
 1.0186641216278076,
 0.0,
 0.16663452982902527,
 0.14928975701332092,
 0.0,
 0.0,
 0.1905912458896637,
 0.0,
 1.4396942853927612,
 0.09840783476829529,
 0.15922053158283234,
 0.7334077954292297,
 0.0,
 0.15451447665691376,
 0.1804513931274414,
 0.0,
 1.644312858581543,
 0.0,
 0.9947700500488281,
 0.9024505019187927,
 0.0,
 0.0,
 0.3060690462589264,
 0.1576310098171234,
 1.8539807796478271,
 0.6755280494689941,
 0.7106196284294128,
 1.4652960300445557,
 0.992569088935852,
 0.604895830154419,
 0.6402587890625,
 0.08659285306930542,
 1.8351320028305054,
 0.6532988548278809,
 0.8272102475166321,
 0.6761192083358765,
 0.9225132465362549,
 0.43156713247299194,
 0.34402182698249817,
 0.3067605197429657,
 1.8281491994857788,
 0.7801331281661987,
 0.6974630355834961,
 0.6179545521736145,


In [21]:
old_list

[0.0,
 1.2464443445205688,
 0.0,
 0.8246548771858215,
 0.0,
 0.0,
 0.0,
 0.38457348942756653,
 0.0,
 1.0052496194839478,
 0.0,
 0.1068667620420456,
 0.07894809544086456,
 0.0,
 0.0,
 0.2448502480983734,
 0.0,
 1.016670823097229,
 0.0,
 0.1663249433040619,
 0.1495649218559265,
 0.0,
 0.0,
 0.18852409720420837,
 0.0,
 1.4383600950241089,
 0.09631088376045227,
 0.1597435623407364,
 0.7316836714744568,
 0.0,
 0.15341518819332123,
 0.18055927753448486,
 0.0,
 1.6475549936294556,
 0.0,
 0.9959424734115601,
 0.9046587347984314,
 0.0,
 0.0,
 0.3053806722164154,
 0.1578250378370285,
 1.8547816276550293,
 0.6739070415496826,
 0.7117723822593689,
 1.4663945436477661,
 0.9914004802703857,
 0.6043463945388794,
 0.6394813060760498,
 0.08423559367656708,
 1.837175726890564,
 0.6514485478401184,
 0.827265202999115,
 0.6778658628463745,
 0.9194700717926025,
 0.43227124214172363,
 0.34267476201057434,
 0.30516502261161804,
 1.828338861465454,
 0.7793667912483215,
 0.6979590654373169,
 0.6195490956306458

In [119]:
new_predictions = predictions
new_predictions[0]['index'] = new_list
new_predictions

[{'bbox': [20, 167, 151, 403],
  'class': 2,
  'index': [0.0,
   1.245349407196045,
   0.0,
   0.8244215250015259,
   0.0,
   0.0,
   0.0,
   0.38313373923301697,
   0.0,
   1.0050204992294312,
   0.0,
   0.10921120643615723,
   0.07870806753635406,
   0.0,
   0.0,
   0.2451404631137848,
   0.0,
   1.0186642408370972,
   0.0,
   0.1666346788406372,
   0.14928963780403137,
   0.0,
   0.0,
   0.19059136509895325,
   0.0,
   1.4396945238113403,
   0.09840771555900574,
   0.1592206060886383,
   0.7334077954292297,
   0.0,
   0.15451441705226898,
   0.1804516613483429,
   0.0,
   1.644312858581543,
   0.0,
   0.9947700500488281,
   0.9024505019187927,
   0.0,
   0.0,
   0.30606919527053833,
   0.1576310694217682,
   1.8539807796478271,
   0.6755284070968628,
   0.7106199264526367,
   1.4652957916259766,
   0.9925693273544312,
   0.6048958897590637,
   0.6402589082717896,
   0.08659298717975616,
   1.835132122039795,
   0.6532987952232361,
   0.8272103667259216,
   0.6761195659637451,
   0.9

In [121]:
import json
db.fs.files.update_one({'_id': _ids[0]}, {'$set': {'metadata.predicitions': json.dumps(new_predictions)}})

In [122]:
index_row = db.fs.files.find_one({'_id': _ids[0]})
saved_list = index_row['metadata']['predictions'][0]['index']
saved_list


[0.0,
 1.2464443445205688,
 0.0,
 0.8246548771858215,
 0.0,
 0.0,
 0.0,
 0.38457348942756653,
 0.0,
 1.0052496194839478,
 0.0,
 0.1068667620420456,
 0.07894809544086456,
 0.0,
 0.0,
 0.2448502480983734,
 0.0,
 1.016670823097229,
 0.0,
 0.1663249433040619,
 0.1495649218559265,
 0.0,
 0.0,
 0.18852409720420837,
 0.0,
 1.4383600950241089,
 0.09631088376045227,
 0.1597435623407364,
 0.7316836714744568,
 0.0,
 0.15341518819332123,
 0.18055927753448486,
 0.0,
 1.6475549936294556,
 0.0,
 0.9959424734115601,
 0.9046587347984314,
 0.0,
 0.0,
 0.3053806722164154,
 0.1578250378370285,
 1.8547816276550293,
 0.6739070415496826,
 0.7117723822593689,
 1.4663945436477661,
 0.9914004802703857,
 0.6043463945388794,
 0.6394813060760498,
 0.08423559367656708,
 1.837175726890564,
 0.6514485478401184,
 0.827265202999115,
 0.6778658628463745,
 0.9194700717926025,
 0.43227124214172363,
 0.34267476201057434,
 0.30516502261161804,
 1.828338861465454,
 0.7793667912483215,
 0.6979590654373169,
 0.6195490956306458